Copyright (c) 2022 Habana Labs, Ltd. an Intel Company.
SPDX-License-Identifier: Apache-2.0


# Inference on Gaudi - Example1

This notebook is used as an example to show inference on the Gaudi Accelerator. This is using a simple model with the MNIST dataset. 

This tutorial will show how to infer an MNIST model using native pytorch api.

Download pretrained model checkpoints from vault

In [ ]:
!wget https://vault.habana.ai/artifactory/misc/inference/mnist/mnist-epoch_20.pth

Import all neccessary dependencies

In [ ]:
import os

#Enable PT_HPU_LAZY_MODE=1
os.environ['PT_HPU_LAZY_MODE'] = '1'

import sys
import torch
import time
import habana_frameworks.torch as ht
import habana_frameworks.torch.core as htcore
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F

Define a simple Net model for MNIST

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1   = nn.Linear(784, 256)
        self.fc2   = nn.Linear(256, 64)
        self.fc3   = nn.Linear(64, 10)
    def forward(self, x):
        out = x.view(-1,28*28)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        out = F.log_softmax(out, dim=1)
        return out

Create the model, and load the pre-trained checkpoint.

In [ ]:
model = Net()
checkpoint = torch.load('mnist-epoch_20.pth')
model.load_state_dict(checkpoint)

Optimize the model for eval, and move the model to Gaudi(hpu)

In [ ]:
model = model.eval()
model = model.to("hpu")

Create a MNIST datasets for evaluation.

In [ ]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])

data_path = './data'
test_kwargs = {'batch_size': 32}
dataset1 = datasets.MNIST(data_path, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(dataset1,**test_kwargs)

Run Inference.
For lazy mode, we need to call mark_step() after each inference to enforce the execution. 

In [ ]:
correct = 0
for batch_idx, (data, label) in enumerate(test_loader):
    data = data.to("hpu")
    output = model(data)
    #In Lazy mode execution, mark_step() need to be added after inference
    htcore.mark_step()
    correct += output.max(1)[1].eq(label).sum()

print('Accuracy: {:.2f}%'.format(100. * correct / (len(test_loader) * 32)))

In [8]:
exit()